<a href="https://colab.research.google.com/github/UWAGroves/Homework-4/blob/main/Homework_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# Link drive & import packages

from google.colab import drive
drive.mount('/content/drive')

filedir = "/content/drive/MyDrive/"

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

#Import data

fname = filedir + "Wcr_GPPdaily.csv"
df = pd.read_csv(fname)

df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,TIMESTAMP,TA_F,SW_IN_F,VPD_F,GPP_NT_VUT_REF
0,19990101,-20.063,72.603,0.413,-0.517364
1,19990102,-12.814,12.358,0.147,-0.094241
2,19990103,-12.625,33.132,0.128,-0.166819
3,19990104,-18.652,93.481,0.263,-0.582301
4,19990105,-20.269,45.502,0.261,-0.568240
...,...,...,...,...,...
5839,20141227,-5.112,22.956,0.497,0.195777
5840,20141228,-12.408,33.332,0.531,0.093025
5841,20141229,-18.204,69.597,0.461,-0.234488
5842,20141230,-22.252,81.959,0.448,-0.315846


In [18]:
# Rename columns

df = df.rename(columns={"TA_F" : "TA", "SW_IN_F" : "SW", "VPD_F" : "VPD", "GPP_NT_VUT_REF" : "GPP"})

# Remove column TIMESTAMP

df = df.drop(columns=["TIMESTAMP"])

df


,TA,SW,VPD,GPP
0,-20.063,72.603,0.413,-0.517364
1,-12.814,12.358,0.147,-0.094241
2,-12.625,33.132,0.128,-0.166819
3,-18.652,93.481,0.263,-0.582301
4,-20.269,45.502,0.261,-0.568240
...,...,...,...,...
5839,-5.112,22.956,0.497,0.195777
5840,-12.408,33.332,0.531,0.093025
5841,-18.204,69.597,0.461,-0.234488
5842,-22.252,81.959,0.448,-0.315846


In [19]:
# Creating interaction terms

df["SW_VPD"] = df["SW"] * df["VPD"]
df["SW_TA"] = df["SW"] * df["TA"]
df["VPD_TA"] = df["VPD"] * df["TA"]

df

,TA,SW,VPD,GPP,SW_VPD,SW_TA,VPD_TA
0,-20.063,72.603,0.413,-0.517364,29.985039,-1456.633989,-8.286019
1,-12.814,12.358,0.147,-0.094241,1.816626,-158.355412,-1.883658
2,-12.625,33.132,0.128,-0.166819,4.240896,-418.291500,-1.616000
3,-18.652,93.481,0.263,-0.582301,24.585503,-1743.607612,-4.905476
4,-20.269,45.502,0.261,-0.568240,11.876022,-922.280038,-5.290209
...,...,...,...,...,...,...,...
5839,-5.112,22.956,0.497,0.195777,11.409132,-117.351072,-2.540664
5840,-12.408,33.332,0.531,0.093025,17.699292,-413.583456,-6.588648
5841,-18.204,69.597,0.461,-0.234488,32.084217,-1266.943788,-8.392044
5842,-22.252,81.959,0.448,-0.315846,36.717632,-1823.751668,-9.968896


In [20]:
# Extract feature matrix (X) and target (y)

X = df[["SW", "VPD", "TA", "SW_VPD", "SW_TA", "VPD_TA"]].values
y = df["GPP"].values

# Add a column of ones for β0 (intercept)

X = np.column_stack((np.ones(X.shape[0]), X))

# Define the MSE loss function

def mse_loss(beta, X, y):
    y_pred = X @ beta
    mse = np.mean((y - y_pred) ** 2)
    return mse

# Initial guess for coefficients (β0, β1, ..., β6)

beta_initial = np.zeros(X.shape[1])

# Minimize the MSE to find optimal coefficients

result = minimize(mse_loss, beta_initial, args=(X, y))

# Extract optimized coefficients

beta_optimal = result.x

print("Optimized Coefficients (β0 to β6):")
print(beta_optimal)

Optimized Coefficients (β0 to β6):
[-9.44070967e+02 -3.24976065e+00 -6.48487923e+02  8.21663933e+01
  1.79235432e+00 -4.56007425e-01  1.35929568e+01]


In [23]:
# Fit the linear regression model

model = LinearRegression()
model.fit(X, y)

# Regression coefficients

beta_0 = model.intercept_
beta_coeffs = model.coef_

print("Intercept (β0):", beta_0)
print("Coefficients (β1 to β6):", beta_coeffs)

Intercept (β0): -944.067201821005
Coefficients (β1 to β6): [ 0.00000000e+00 -3.24914968e+00 -6.48574173e+02  8.21699813e+01
  1.79249151e+00 -4.56050357e-01  1.35954025e+01]


In [24]:
# SciPy R2

y_pred_scipy = X @ beta_optimal
r2_scipy = r2_score(y, y_pred_scipy)

# Sklearn R2
r2_sklearn = model.score(X, y)

print("R-squared (SciPy Optimized Model):", r2_scipy)
print("R-squared (Sklearn LinearRegression Model):", r2_sklearn)

R-squared (SciPy Optimized Model): 0.013582700734535713
R-squared (Sklearn LinearRegression Model): 0.013582700933255087
